# Count connected tracks inside a cone

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('DS_2_train.csv', index_col=0)
df_test = pd.read_csv('DS_2_test.csv', index_col=0)

/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
zl = np.unique(df['Z'])
print(zl)
zmin = np.min(zl)
zmax = np.max(zl)
bw = 1000
dz = np.diff(zl)[0]
xl = np.arange(27700, 74000, bw)
yl = np.arange(3300, 77900, bw)

[     0.   1293.   2586.   3879.   5172.   6465.   7758.   9051.  10344.
  11637.  12930.  14223.  15516.  16809.  18102.  19395.  20688.  21981.
  23274.  24567.  25860.  27153.  28446.  29739.  31032.  32325.  33618.
  34911.  36204.  37497.  38790.  40083.  41376.  42669.  43962.  45255.
  46548.  47841.  49134.  50427.  51720.  53013.  54306.  55599.  56892.
  58185.  59478.  60771.  62064.  63357.  64650.  65943.  67236.  68529.
  69822.  71115.  72408.]


In [4]:
bs = np.array([1,3,1])
kr1 = np.outer(bs, np.outer(bs, bs)).reshape(3,3,3)
from scipy.ndimage import convolve

In [5]:
dd = df.query("brick_number==57")
cnt, bnd = np.histogramdd(dd[["X", "Y", "Z"]].values, bins=(xl, yl, np.append(zl, zmax+2*dz)))
dd['cnt'] = cnt[(dd["X"].values.astype(int) - 27700)//bw, 
                (dd["Y"].values.astype(int) - 3300)//bw, 
                dd["Z"].values.astype(int)//1293]

/fefs/disk/usr100/gentoo/usr/lib64/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [6]:
def hiss(dd, n):
    cnt, bnd = np.histogramdd(dd[["X", "Y", "Z"]].values, bins=(xl, yl, np.append(zl, zmax+2*dz)))
    for i in np.arange(n):
        cnt = convolve(cnt, kr1)
    dd['cnt'] = cnt[(dd["X"].values.astype(int) - 27700)//bw, 
                    (dd["Y"].values.astype(int) - 3300)//bw, 
                    dd["Z"].values.astype(int)//1293]
    return dd

In [7]:
def hiss_n(n):
    rename = {"cnt":"cnt{}".format(n), "dback":"cnt{}".format(n)}
    xdf = df.groupby("brick_number").apply(hiss, n)
    xdfo = xdf[['cnt']].rename(columns=rename)
    xdfo.to_hdf("hiss-r{}.h5".format(n), "train")
    xdft = df_test.groupby("brick_number").apply(hiss, n)
    xdfot = xdft[['cnt']].rename(columns=rename)
    xdfot.to_hdf("hiss-r{}.h5".format(n), "test")
    return n

In [8]:
from concurrent.futures import ProcessPoolExecutor

In [9]:
with ProcessPoolExecutor(max_workers=6) as executor:
    rst = list(executor.map(hiss_n, range(6)))